In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow import keras
from PIL import Image
from collections import Counter

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F


In [2]:
import xmltodict
import cv2
from PIL import Image

img_width = 224
img_height = 224

def zoom_at(img, x, y, zoom):
	w, h = img.size
	zoom2 = zoom * 2
	img = img.crop((x - w / zoom2, y - h / zoom2, 
					x + w / zoom2, y + h / zoom2))
	return img.resize((w, h), Image.LANCZOS)

def get_number_of_positives_and_negatives(past, seq):
	dir_imgs = past + str(seq) + "/imgs/"
	positives = 0
	negatives = 0

	with open(past + str(seq) + "/annotations.xml", "rb") as org_file:
		dict_file = xmltodict.parse(org_file)

	arr_plants = dict_file["annotations"]["image"]

	for file in arr_plants:
		try:
			name_img = file["@name"]
			x_min = file["box"]
			positives += 1

		except Exception as e:
			negatives += 1
	return positives, negatives


def get_arrays_from_past_and_seq(past, seq):
	x_array = []
	y_array = []
	dir_imgs = past + str(seq) + "/imgs/"

	with open(past + str(seq) + "/annotations.xml", "rb") as org_file:
		dict_file = xmltodict.parse(org_file)

	arr_plants = dict_file["annotations"]["image"]

	for file in arr_plants:
		try:
			name_img = file["@name"]
			x_min = file["box"]
			img = Image.open(dir_imgs + name_img)
			img = img.resize((img_width, img_height)) # 0.3 = 576x324 --- 0.2 = 384x216 --- 0.15 = 288x162
			x_array.append(img)
			y_array.append(1)

		except:
			name_img = file["@name"]
			img = Image.open(dir_imgs + name_img)
			img = img.resize((img_width, img_height))
			rotated_image1 = img.rotate(180)
			rotated_image1 = zoom_at(rotated_image1, 960, 540, 1.5)
			# rotated_image1 = rotated_image1.resize((img_width, img_height))
			# img.show()
			# rotated_image1.show()
			x_array.append(rotated_image1)
			y_array.append(0)
			x_array.append(img)
			y_array.append(0)

	return x_array, y_array

past_and_last_seq_0 = ("RumexWeeds/20210806_hegnstrup/seq", 17)
past_and_last_seq_1 = ("RumexWeeds/20210806_stengard/seq", 20)
past_and_last_seq_2 = ("RumexWeeds/20210807_lundholm/seq", 28)
past_and_last_seq_3 = ("RumexWeeds/20210908_lundholm/seq", 13)
past_and_last_seq_4 = ("RumexWeeds/20211006_stengard/seq", 15)

x_train = []
y_train = []

x_test = []
y_test = []

train = past_and_last_seq_2
test = past_and_last_seq_0
for i in range(train[1]+1):
	x_aux, y_aux = get_arrays_from_past_and_seq(train[0], i)
	x_train.extend(x_aux)
	y_train.extend(y_aux)

for i in range(test[1]+1):
	x_aux, y_aux = get_arrays_from_past_and_seq(test[0], i)
	x_test.extend(x_aux)
	y_test.extend(y_aux)

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)


total_pos = 0
total_neg = 0
for i in range(train[1]+1):
	p, n = get_number_of_positives_and_negatives(train[0], i)
	total_pos += p
	total_neg += n
	#print(i, " = ", p, "  ", n)
print(train[0])
print("Total treino = ", len(y_train))
print("Positives: ", total_pos)
print("Negatives: ", total_neg, "\n")


total_pos = 0
total_neg = 0
for i in range(test[1]+1):
	p, n = get_number_of_positives_and_negatives(test[0], i)
	total_pos += p
	total_neg += n
	#print(i, " = ", p, "  ", n)
print(test[0])
print("Total teste = ", len(y_test))
print("Positives: ", total_pos)
print("Negatives: ", total_neg, "\n")

print(x_train.shape)
print(x_test.shape)
# (580, 1200, 1920, 3)

RumexWeeds/20210807_lundholm/seq
Total treino =  1470
Positives:  786
Negatives:  342 

RumexWeeds/20210806_hegnstrup/seq
Total teste =  706
Positives:  454
Negatives:  126 

(1470, 224, 224, 3)
(706, 224, 224, 3)


In [3]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers import concatenate

In [4]:

def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
	# Input: 
	# - f1: number of filters of the 1x1 convolutional layer in the first path
	# - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
	# - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
	# - f4: number of filters of the 1x1 convolutional layer in the fourth path

	# 1st path:
	path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

	# 2nd path
	path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
	path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

	# 3rd path
	path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
	path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

	# 4th path
	path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
	path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

	output_layer = concatenate([path1, path2, path3, path4], axis = -1)

	return output_layer

def GoogLeNet():
	# input layer 
	input_layer = Input(shape = (img_width, img_height, 3))

	# convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
	X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)

	# max-pooling layer: pool_size = (3,3), strides = 2
	X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

	# convolutional layer: filters = 64, strides = 1
	X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

	# convolutional layer: filters = 192, kernel_size = (3,3)
	X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

	# max-pooling layer: pool_size = (3,3), strides = 2
	X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

	# 1st Inception block
	X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

	# 2nd Inception block
	X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

	# max-pooling layer: pool_size = (3,3), strides = 2
	X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

	# 3rd Inception block
	X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

	# Extra network 1:
	X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
	X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
	X1 = Flatten()(X1)
	X1 = Dense(1024, activation = 'relu')(X1)
	X1 = Dropout(0.7)(X1)
	X1 = Dense(2, activation = 'softmax')(X1)

	
	# 4th Inception block
	X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

	# 5th Inception block
	X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

	# 6th Inception block
	X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

	# Extra network 2:
	X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
	X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
	X2 = Flatten()(X2)
	X2 = Dense(1024, activation = 'relu')(X2)
	X2 = Dropout(0.7)(X2)
	X2 = Dense(2, activation = 'softmax')(X2)
	
	
	# 7th Inception block
	X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
											f3_conv5 = 128, f4 = 128)

	# max-pooling layer: pool_size = (3,3), strides = 2
	X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

	# 8th Inception block
	X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

	# 9th Inception block
	X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

	# Global Average pooling layer 
	X = GlobalAveragePooling2D(name = 'GAPL')(X)

	# Dropoutlayer 
	X = Dropout(0.4)(X)

	# output layer 
	X = Dense(2, activation = 'softmax')(X)
	
	# model
	model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

	return model

model = GoogLeNet() 
model.summary()

Model: "GoogLeNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 109, 109,  │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 54, 54,    │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 54, 54,    │    110,784 │ conv2d_1[0][0]    │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 26, 26,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 26, 26,    │     18,528 │ max_pooling2d_1[… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 26, 26,    │      3,088 │ max_pooling2d_1[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 26, 26,    │          0 │ max_pooling2d_1[… │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 26, 26,    │     12,352 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 26, 26,    │    110,720 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 26, 26,    │     12,832 │ conv2d_6[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 26, 26,    │      6,176 │ max_pooling2d_2[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 26, 26,    │          0 │ conv2d_3[0][0],   │
│ (Concatenate)       │ 256)              │            │ conv2d_5[0][0],   │
│                     │                   │            │ conv2d_7[0][0],   │
│                     │                   │            │ conv2d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 26, 26,    │     32,896 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 26, 26,    │      8,224 │ concatenate[0][0] │
│                     │ 32)               │            │                 

 Total params: 8,474,422 (32.33 MB)

 Trainable params: 8,474,422 (32.33 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np

my_dict = {
	str(str(img_width) + "_Duplicated_AC"): [],
	"EPOCH":[],
	"SENS": [],
	"ESP": []
}

for i in range(5, 15):
	print("Onde estou -> ", i)
	model = GoogLeNet() 
	model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', 'accuracy', 'accuracy'])
	model.fit(x_train, y_train, epochs=i)

	Y_test = []
	answer = model.predict(x_test)
	for elem in y_test:
		Y_test.append(elem)
		
	answer_2 = np.argmax(answer[0], axis=1)
	TN, FP, FN, TP = confusion_matrix(Y_test, answer_2).ravel()
	my_dict[str(str(img_width) + "_Duplicated_AC")].append(accuracy_score(Y_test, answer_2))
	my_dict["EPOCH"].append(i)
	my_dict["SENS"].append((TP/(TP+FN)))
	my_dict["ESP"].append((TN/(FP+TN)))

import pandas as pd

my_dict = pd.DataFrame(my_dict)

my_dict.to_csv("Result_"+ str(img_width) +"_Duplicated.csv", index=False)

Onde estou ->  5
Epoch 1/5
46/46 ━━━━━━━━━━━━━━━━━━━━ 52s 843ms/step - dense_9_accuracy: 0.7186 - loss: 1.6890
Epoch 2/5
46/46 ━━━━━━━━━━━━━━━━━━━━ 38s 832ms/step - dense_9_accuracy: 0.7701 - loss: 0.6191
Epoch 3/5
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 841ms/step - dense_9_accuracy: 0.6560 - loss: 1.2834
Epoch 4/5
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 845ms/step - dense_9_accuracy: 0.7596 - loss: 0.5921
Epoch 5/5
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 846ms/step - dense_9_accuracy: 0.7656 - loss: 0.4861
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 262ms/step
Onde estou ->  6
Epoch 1/6


C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


46/46 ━━━━━━━━━━━━━━━━━━━━ 51s 862ms/step - dense_14_accuracy: 0.6832 - loss: 0.7727
Epoch 2/6
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 858ms/step - dense_14_accuracy: 0.7586 - loss: 0.4895
Epoch 3/6
46/46 ━━━━━━━━━━━━━━━━━━━━ 40s 860ms/step - dense_14_accuracy: 0.7732 - loss: 0.4731
Epoch 4/6
46/46 ━━━━━━━━━━━━━━━━━━━━ 40s 869ms/step - dense_14_accuracy: 0.7599 - loss: 0.4793
Epoch 5/6
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 854ms/step - dense_14_accuracy: 0.7700 - loss: 0.4771
Epoch 6/6
46/46 ━━━━━━━━━━━━━━━━━━━━ 40s 860ms/step - dense_14_accuracy: 0.7783 - loss: 0.4730
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 255ms/step
Onde estou ->  7
Epoch 1/7


C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


46/46 ━━━━━━━━━━━━━━━━━━━━ 50s 852ms/step - dense_19_accuracy: 0.7183 - loss: 1.6721
Epoch 2/7
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 852ms/step - dense_19_accuracy: 0.7532 - loss: 0.4888
Epoch 3/7
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 852ms/step - dense_19_accuracy: 0.7569 - loss: 0.4929
Epoch 4/7
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 852ms/step - dense_19_accuracy: 0.7648 - loss: 0.4815
Epoch 5/7
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 852ms/step - dense_19_accuracy: 0.7549 - loss: 0.4796
Epoch 6/7
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 845ms/step - dense_19_accuracy: 0.7597 - loss: 0.4877
Epoch 7/7
46/46 ━━━━━━━━━━━━━━━━━━━━ 39s 856ms/step - dense_19_accuracy: 0.7605 - loss: 0.4811
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 260ms/step
Onde estou ->  8
Epoch 1/8


...

46/46 ━━━━━━━━━━━━━━━━━━━━ 71s 921ms/step - dense_44_accuracy: 0.6792 - loss: 2.4707
Epoch 2/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 882ms/step - dense_44_accuracy: 0.7735 - loss: 0.4741
Epoch 3/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 903ms/step - dense_44_accuracy: 0.7784 - loss: 0.4674
Epoch 4/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 892ms/step - dense_44_accuracy: 0.7742 - loss: 0.4622
Epoch 5/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 883ms/step - dense_44_accuracy: 0.7514 - loss: 0.4922
Epoch 6/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 895ms/step - dense_44_accuracy: 0.7567 - loss: 0.4838
Epoch 7/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 880ms/step - dense_44_accuracy: 0.7602 - loss: 0.4959
Epoch 8/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 888ms/step - dense_44_accuracy: 0.5797 - loss: 0.7179
Epoch 9/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 893ms/step - dense_44_accuracy: 0.6379 - loss: 1.1733
Epoch 10/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 42s 909ms/step - dense_44_accuracy: 0.7355 - loss: 0.5821
Epoch 11/12
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 892ms/s

C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


46/46 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - dense_49_accuracy: 0.6745 - loss: 2.8045
Epoch 2/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 43s 925ms/step - dense_49_accuracy: 0.7400 - loss: 0.5709
Epoch 3/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 882ms/step - dense_49_accuracy: 0.7567 - loss: 0.4898
Epoch 4/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 40s 878ms/step - dense_49_accuracy: 0.7699 - loss: 0.4774
Epoch 5/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 40s 874ms/step - dense_49_accuracy: 0.7593 - loss: 0.4826
Epoch 6/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - dense_49_accuracy: 0.7717 - loss: 0.4715
Epoch 7/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - dense_49_accuracy: 0.7558 - loss: 0.4826
Epoch 8/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - dense_49_accuracy: 0.7641 - loss: 0.4693
Epoch 9/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 43s 924ms/step - dense_49_accuracy: 0.7875 - loss: 0.4623
Epoch 10/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 45s 985ms/step - dense_49_accuracy: 0.7703 - loss: 0.4759
Epoch 11/13
46/46 ━━━━━━━━━━━━━━━━━━━━ 45s 981ms/step - dense_

C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


46/46 ━━━━━━━━━━━━━━━━━━━━ 59s 948ms/step - dense_54_accuracy: 0.6695 - loss: 0.9714
Epoch 2/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 44s 958ms/step - dense_54_accuracy: 0.7204 - loss: 0.4983
Epoch 3/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 899ms/step - dense_54_accuracy: 0.7464 - loss: 0.5017
Epoch 4/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 40s 877ms/step - dense_54_accuracy: 0.7651 - loss: 0.4670
Epoch 5/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 882ms/step - dense_54_accuracy: 0.7523 - loss: 0.4940
Epoch 6/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 42s 905ms/step - dense_54_accuracy: 0.7456 - loss: 0.4856
Epoch 7/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 42s 905ms/step - dense_54_accuracy: 0.7704 - loss: 0.4750
Epoch 8/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 42s 903ms/step - dense_54_accuracy: 0.7447 - loss: 0.4902
Epoch 9/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 898ms/step - dense_54_accuracy: 0.7644 - loss: 0.4702
Epoch 10/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 42s 916ms/step - dense_54_accuracy: 0.7637 - loss: 0.4769
Epoch 11/14
46/46 ━━━━━━━━━━━━━━━━━━━━ 42s 906ms/s